In [2]:
import fedmriapp.mristrategy.average as average
from fedmriapp.models.models import get_custom_model
from fedmriapp.client_app import get_data_loader
from skimage.morphology import closing, footprint_rectangle, binary_erosion, remove_small_objects, disk
from skimage.measure import label, regionprops
import numpy as np


2025-02-18 10:26:05,066 - INFO - PyTorch version 2.5.1 available.


In [5]:
partition_id = 11


# Initialize model with fixed seed
net = get_custom_model()
#print("Client", partition_id, "loaded")
# string_to_write = f"Client {partition_id} loaded\n"
# with open("fedmriapp/results/client_ids.txt", "a") as f:
#     f.write(string_to_write)
trainloader =  get_data_loader(partition_id, batch_size=32, noise_type=None, noise_params=None)
trainloader_noisy = get_data_loader(partition_id, batch_size=32, noise_type="rician", noise_params={"std":0.8}) 

mean_snr, mean_cnr = average.calculate_dataset_snr_cnr(trainloader, aggregation_method="median")
mean_snr_noisy, mean_cnr_noisy = average.calculate_dataset_snr_cnr(trainloader_noisy, aggregation_method="median")

print("Client", partition_id, "SNR:", mean_snr)
print("Client", partition_id, "CNR:", mean_cnr)

print("Client", partition_id, "SNR noisy:", mean_snr_noisy)
print("Client", partition_id, "CNR noisy:", mean_cnr_noisy)

2025-02-18 10:44:20,570 - INFO - CustomCNNLight initialized with seed 42
2025-02-18 10:44:20,577 - INFO - Successfully created custom model with seed 42


Distribution used: iid
URL: http://172.16.5.71:5000/partitions/11
Distribution used: iid
URL: http://172.16.5.71:5000/partitions/11
Client 11 SNR: 5.1899695
Client 11 CNR: 4.732081627792917
Client 11 SNR noisy: 1.5719783
Client 11 CNR noisy: 2.1104874128546776


In [3]:
# def calculate_cnr(image):
#     """
#     Calculate Contrast-to-Noise Ratio (CNR) for a brain MRI image.
    
#     Args:
#     - image: 2D numpy array of the MRI image.
    
#     Returns:
#     - CNR value.
#     """
#     # Step 1: Binarize the image using Otsu's thresholding
#     try:
#         thresh = average.threshold_otsu(image)
#     except ValueError:
#         return 0.0
    
    
#     # Use a 3x3 rectangle structuring element for closing
#     bw = closing(image > thresh, footprint_rectangle((3, 3)))  # Replaced square(3) with rectangle(3,3)
    
#     # Step 2: Label regions and keep the largest one (brain)
#     labeled = label(bw)
#     regions = regionprops(labeled)
    
#     if not regions:
#         return 0.0
    
#     largest_region = max(regions, key=lambda x: x.area)
#     brain_mask = labeled == largest_region.label
    
#     # Remove small holes/objects
#     brain_mask = remove_small_objects(brain_mask, min_size=50)
    
#     # Step 3: Erode the brain mask (using rectangle instead of square)
#     eroded_mask = binary_erosion(brain_mask, footprint_rectangle(3, 3))  # Updated here
#     if np.sum(eroded_mask) == 0:
#         eroded_mask = brain_mask
    
#     # Step 4: Define background as outer border (exclude brain)
#     border_width = 10
#     rows, cols = image.shape
#     border_mask = np.zeros_like(image, dtype=bool)
#     border_mask[:border_width, :] = True
#     border_mask[-border_width:, :] = True
#     border_mask[:, :border_width] = True
#     border_mask[:, -border_width:] = True
    
#     background = np.logical_and(border_mask, ~brain_mask)
#     if np.sum(background) == 0:
#         background = border_mask
    
#     # Extract regions and compute CNR
#     signal_region = image[eroded_mask]
#     background_region = image[background]
    
#     if signal_region.size == 0 or background_region.size == 0:
#         return 0.0
    
#     mean_signal = np.mean(signal_region)
#     mean_background = np.mean(background_region)
#     std_background = np.std(background_region)
    
#     cnr = (mean_signal - mean_background) / std_background if std_background != 0 else 0.0
#     return cnr

In [ ]:
import numpy as np


import matplotlib.pyplot as plt

# Get a batch of training data
dataiter = iter(trainloader)
print("dataiter", next(dataiter))
element = next(dataiter)
images, labels = element['image'], element['label']

# Convert images to numpy for plotting
image_original = images[10].numpy()
image = np.transpose(image_original, (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)

# Plot the image
plt.imshow(image, cmap='gray')
plt.title(f'Label: {labels[0].item()}')
plt.show()

In [ ]:
average.calculate_cnr(image)

In [6]:
thresh = average.threshold_otsu(image_original)


In [ ]:
thresh

In [8]:
footprint = footprint_rectangle((3,3))

In [ ]:
footprint

In [10]:
image_original = np.squeeze(image_original)

In [11]:
bw = closing(image_original > thresh, footprint)

In [ ]:

bw

In [13]:
import numpy as np
import scipy.stats as stats
from skimage import filters
from scipy.ndimage import gaussian_filter

def compute_cnr(image, method='otsu', sigma=1.0):
    """
    Compute Contrast-to-Noise Ratio (CNR) for an MRI image using adaptive segmentation.
    
    Parameters:
    -----------
    image : numpy.ndarray
        2D or 3D MRI image
    method : str, optional
        Thresholding method to define regions
        Options: 'otsu', 'mean', 'median'
    sigma : float, optional
        Gaussian filter sigma for noise estimation
    
    Returns:
    --------
    dict containing:
    - CNR value
    - Signal intensities of regions
    - Noise estimation
    """
    # Ensure input is a numpy array
    image = np.array(image, dtype=np.float64)
    
    # Noise estimation using Gaussian filtering
    noise_image = gaussian_filter(image, sigma=sigma)
    noise_std = np.std(noise_image - image)
    
    # Region segmentation based on selected method
    if method == 'otsu':
        # Otsu's thresholding for adaptive segmentation
        threshold = filters.threshold_otsu(image)
        region1 = image[image > threshold]
        region2 = image[image <= threshold]
    elif method == 'mean':
        # Mean-based segmentation
        mean_intensity = np.mean(image)
        region1 = image[image > mean_intensity]
        region2 = image[image <= mean_intensity]
    elif method == 'median':
        # Median-based segmentation
        median_intensity = np.median(image)
        region1 = image[image > median_intensity]
        region2 = image[image <= median_intensity]
    else:
        raise ValueError("Invalid method. Choose 'otsu', 'mean', or 'median'.")
    
    # Compute signal intensities
    signal1_mean = np.mean(region1)
    signal2_mean = np.mean(region2)
    
    # Compute CNR
    cnr = abs(signal1_mean - signal2_mean) / noise_std
    
    return {
        'cnr': cnr,
        'signal1_mean': signal1_mean,
        'signal2_mean': signal2_mean,
        'noise_std': noise_std
    }